# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 22 2022 12:05PM,253681,12,EVO1016,"Evolution Protection, LLC",Released
1,Dec 22 2022 12:05PM,253680,19,8935423,"Graystone, LLC",Released
2,Dec 22 2022 12:05PM,253680,19,8935425,"Graystone, LLC",Released
3,Dec 22 2022 11:35AM,253679,10,SOTAH0000863,"Nextsource Biotechnology, LLC",Released
4,Dec 22 2022 11:35AM,253679,10,SONSB0002063,"Nextsource Biotechnology, LLC",Released
5,Dec 22 2022 11:31AM,253674,10,0086265516,ISDIN Corporation,Released
6,Dec 22 2022 11:31AM,253674,10,0086265524,ISDIN Corporation,Released
7,Dec 22 2022 11:31AM,253674,10,0086265525,ISDIN Corporation,Released
8,Dec 22 2022 11:31AM,253674,10,0086265547,ISDIN Corporation,Released
9,Dec 22 2022 11:31AM,253674,10,0086265561,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,253677,Released,1
24,253678,Released,1
25,253679,Released,2
26,253680,Released,2
27,253681,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253677,NaN,1.0
253678,NaN,1.0
253679,NaN,2.0
253680,NaN,2.0
253681,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253622,0.0,1.0
253624,0.0,1.0
253625,0.0,1.0
253628,0.0,1.0
253629,1.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253622,0,1
253624,0,1
253625,0,1
253628,0,1
253629,1,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253622,0,1
1,253624,0,1
2,253625,0,1
3,253628,0,1
4,253629,1,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253622,,1
1,253624,,1
2,253625,,1
3,253628,,1
4,253629,1,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 22 2022 12:05PM,253681,12,"Evolution Protection, LLC"
1,Dec 22 2022 12:05PM,253680,19,"Graystone, LLC"
3,Dec 22 2022 11:35AM,253679,10,"Nextsource Biotechnology, LLC"
5,Dec 22 2022 11:31AM,253674,10,ISDIN Corporation
10,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation
11,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation
12,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation
42,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation
49,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc."
55,Dec 22 2022 11:08AM,253672,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 22 2022 12:05PM,253681,12,"Evolution Protection, LLC",,1
1,Dec 22 2022 12:05PM,253680,19,"Graystone, LLC",,2
2,Dec 22 2022 11:35AM,253679,10,"Nextsource Biotechnology, LLC",,2
3,Dec 22 2022 11:31AM,253674,10,ISDIN Corporation,,5
4,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,,1
5,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,,1
6,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,,30
7,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,,7
8,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",,6
9,Dec 22 2022 11:08AM,253672,10,Bio-PRF,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 12:05PM,253681,12,"Evolution Protection, LLC",1,
1,Dec 22 2022 12:05PM,253680,19,"Graystone, LLC",2,
2,Dec 22 2022 11:35AM,253679,10,"Nextsource Biotechnology, LLC",2,
3,Dec 22 2022 11:31AM,253674,10,ISDIN Corporation,5,
4,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,
5,Dec 22 2022 11:30AM,253677,10,ISDIN Corporation,1,
6,Dec 22 2022 11:24AM,253675,10,ISDIN Corporation,30,
7,Dec 22 2022 11:24AM,253673,10,ISDIN Corporation,7,
8,Dec 22 2022 11:23AM,253676,10,"Methapharm, Inc.",6,
9,Dec 22 2022 11:08AM,253672,10,Bio-PRF,7,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 12:05PM,253681,12,"Evolution Protection, LLC",1,
1,Dec 22 2022 12:05PM,253680,19,"Graystone, LLC",2,
2,Dec 22 2022 11:35AM,253679,10,"Nextsource Biotechnology, LLC",2,
3,Dec 22 2022 11:31AM,253674,10,ISDIN Corporation,5,
4,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 12:05PM,253681,12,"Evolution Protection, LLC",1,NaN
1,Dec 22 2022 12:05PM,253680,19,"Graystone, LLC",2,NaN
2,Dec 22 2022 11:35AM,253679,10,"Nextsource Biotechnology, LLC",2,NaN
3,Dec 22 2022 11:31AM,253674,10,ISDIN Corporation,5,NaN
4,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 22 2022 12:05PM,253681,12,"Evolution Protection, LLC",1,0.0
1,Dec 22 2022 12:05PM,253680,19,"Graystone, LLC",2,0.0
2,Dec 22 2022 11:35AM,253679,10,"Nextsource Biotechnology, LLC",2,0.0
3,Dec 22 2022 11:31AM,253674,10,ISDIN Corporation,5,0.0
4,Dec 22 2022 11:30AM,253678,10,ISDIN Corporation,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5073173,116,1.0
12,253681,1,0.0
17,760877,3,0.0
19,760964,38,0.0
21,253668,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5073173,116,1.0
1,12,253681,1,0.0
2,17,760877,3,0.0
3,19,760964,38,0.0
4,21,253668,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,116,1.0
1,12,1,0.0
2,17,3,0.0
3,19,38,0.0
4,21,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,12,Released,1.0
2,17,Released,3.0
3,19,Released,38.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,17,19,21
Status,,,,,
Executing,1.0,0.0,0.0,0.0,0.0
Released,116.0,1.0,3.0,38.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,17,19,21
0,Executing,1.0,0.0,0.0,0.0,0.0
1,Released,116.0,1.0,3.0,38.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,17,19,21
0,Executing,1.0,0.0,0.0,0.0,0.0
1,Released,116.0,1.0,3.0,38.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()